# Main notebook 
## used to train different model parameters and using different losses on google colab

In [ ]:
import tensorflow as tf
import os
import sys

import numpy as np

from tqdm import tqdm
from itertools import chain

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import os
import matplotlib.image as mpimg
from PIL import Image

from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers import Dropout
from keras import optimizers
from keras.layers import BatchNormalization


from numpy import load
from numpy import asarray
from numpy import savez_compressed

import tensorflow as tf
from tensorflow import keras

import re


import sys

from helpers import *

from smooth_tiled_predictions import cheap_tiling_prediction


print(tf.version.VERSION)

directoryOfProject = 'drive/MyDrive/Project2ML/'

data_dir = directoryOfProject + 'trainingWithGenerator/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/'

test_dir = directoryOfProject + 'test/'
test_dir_image = test_dir + 'images/test_'
test_dir_groundtruth = test_dir + 'groundtruth/test_'

originalDataImage = directoryOfProject + 'training/' + 'images/'
originalDataGrountruth = directoryOfProject + 'training/' + 'groundtruth/'

IMAGE_OPEN = os.listdir(train_data_filename)


IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
PIXEL_DEPTH = 255
SEED = 66478
IMG_WIDTH    = 400
IMG_HEIGHT   = 400

IMG_PATCH_SIZE = 16

NUMBER_OF_EXTRA_IMAGES_PER_IMAGE = 40

NUMBER_OF_FILE_TO_SEARSH_TRESHOLD = 200

VALIDATIONSPLIT = 0.2
BATCH_SIZE = 32
EPOCHS = 100

NEED_TO_GENERATE_NEW_IMAGES = False #If the new images created from the 100 given images don't exist
COMPRESSED_IMAGE_EXIST = False #If you have loaded the images before
PREDICTION_IMAGE_EXIST = False #If you have created/loaded the prediction using the model 

BINARY_AND_JACCARD = False
BINARY_AND_DICE = False
BINARY_CROSS_ENTROPY = True

NEED_TO_CREATE_MODEL = False #If the model hasn't been trained/Created before



foreground_threshold = 0.51 #Value of the threshold for the AICrown submission

if (BINARY_AND_JACCARD):
  extraDirectory = 'jaccard'
if (BINARY_AND_DICE):
  extraDirectory = 'dice'
if (BINARY_CROSS_ENTROPY):
  extraDirectory = 'binnary'







# Generate Images to augment dataset

In [ ]:

def generate_images(save_to, imgs_number):
    '''
    Image generating function, it will save generated images in a given folder.
    :param save_to: folder name where to save the images
    :param n: number of images generated per source image ( n= 10 will result in 100*10=1000 images)
    '''

    source_images = os.listdir(train_data_path)
    source_groundtruth = os.listdir(train_labels_path)
    BATCH_SIZE = 32

    if not path.exists('../data/{}'.format(save_to)):
        os.mkdir('../data/{}'.format(save_to))
    if not path.exists('../data/{}/images'.format(save_to)):
        os.mkdir('../data/{}/images'.format(save_to))
    else:
        if (len(os.listdir('../data/{}/images'.format(save_to)))==100*imgs_number):
            print("Existing images found!")
            return 0

    if not path.exists('../data/{}/labels'.format(save_to)):
        os.mkdir('../data/{}/labels'.format(save_to))
    new_img_folder = '../data/{}/images'.format(save_to)
    new_gt_folder = '../data/{}/labels'.format(save_to)

    for n, id_ in tqdm(enumerate(source_images), total=len(source_images)):
        img = imread(train_data_path + id_ )[:,:,:IMG_CHANNELS]
        img = resize(img, (IMG_SIZE, IMG_SIZE), mode='constant', preserve_range=True)
        img = np.expand_dims(img, axis=0)
        mask = np.zeros((IMG_SIZE, IMG_SIZE, 1), dtype=np.bool)
        mask_ = imread(train_labels_path + id_)
        mask_ = np.expand_dims(resize(mask_, (IMG_SIZE, IMG_SIZE), mode='constant',
                                            preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
        mask = np.expand_dims(mask, axis=0)

        # This is the image data genenator where you can tweak the parameters
        datagen = ImageDataGenerator(
        rotation_range=360,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.6,
        fill_mode="reflect",
        horizontal_flip=True,
        vertical_flip=True,
        )
        imageGenerated = datagen.flow(
        img,
        y=mask,
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=SEED,
        save_to_dir=new_img_folder,
        save_prefix=str(n) ,
        save_format="png",
        )
        groundTruthGenerated = datagen.flow(
        mask,
        y=mask,
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=SEED,
        save_to_dir=new_gt_folder,
        save_prefix= str(n),
        save_format="png",
        )
        totalgenerated=0

        for image in imageGenerated:
            totalgenerated+=1
            if (totalgenerated >= imgs_number):
                totalgenerated=0
                break 

        for image in groundTruthGenerated:   
            totalgenerated+=1
            if (totalgenerated >= imgs_number):
                totalgenerated=0
                break 
                
                
if (NEED_TO_GENERATE_NEW_IMAGES):
    IMAGE_OPEN_original = os.listdir(originalDataImage)
    generate_images(train_data_filename, NUMBER_OF_EXTRA_IMAGES_PER_IMAGE)



In [ ]:
# This part is to load the already existing images with their groundtruth. 
# If it is already done, you can load the compressed version. 

X = np.zeros((len(IMAGE_OPEN), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y = np.zeros((len(IMAGE_OPEN), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

if (COMPRESSED_IMAGE_EXIST == False):
  for n, id_ in tqdm(enumerate(IMAGE_OPEN), total=len(IMAGE_OPEN)):
      path = data_dir
      img = imread(train_data_filename + id_ )[:,:,:IMG_CHANNELS]
      img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
      X[n] = img
      mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
      mask_ = imread(train_labels_filename + id_)
      mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',
                                        preserve_range=True), axis=-1)
      mask = np.maximum(mask, mask_)
      Y[n] = mask
  x_train=X
  y_train=Y
  savez_compressed(data_dir +'/X.npz', x_train)
  savez_compressed(data_dir +'/Y.npz', y_train)
else:
  x_train = np.load(data_dir +'/X.npz')['arr_0']
  y_train = np.load(data_dir +'/Y.npz')['arr_0']





# Different losses

In [ ]:
def jaccard_loss(y_true, y_pred,  smooth=1):
    '''
    jaccard loss function
    :param y_true: true labels 
    :param y_pred: predicted labels
    :return: computed loss
    '''
    y_true_int = tf.where(y_true==True, 1., 0.)
    y_true_f = tf.keras.backend.flatten(y_true_int)
    y_pred_f = tf.keras.backend.flatten(y_pred)

    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    jaccard = (intersection +  smooth) / (tf.keras.backend.sum(y_true_f)   +  tf.keras.backend.sum(y_pred_f) - intersection + smooth)

    return 1-jaccard

def dice_loss(y_true, y_pred, smooth = 1):
    '''
    dice loss function
    :param y_true: true labels 
    :param y_pred: predicted labels
    :return: computed loss
    '''
    y_true_int = tf.where(y_true==True, 1., 0.)
    y_true_f = tf.keras.backend.flatten(y_true_int)
    y_pred_f = tf.keras.backend.flatten(y_pred)

    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f)   +  tf.keras.backend.sum(y_pred_f) + smooth)

    return 1-dice

def jaccard_and_binary_crossentropy(y_true, y_pred):
    '''
    jaccard and binary crossentropy loss function
    :param y_true: true labels 
    :param y_pred: predicted labels
    :return: computed loss
    '''
    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + jaccard_loss(y_true, y_pred)

def dice_and_binary_crossentropy(y_true, y_pred):
    '''
    dice and binary crossentropy loss function
    :param y_true: true labels 
    :param y_pred: predicted labels
    :return: computed loss
    '''
    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)


In [ ]:
#We pick the loss we want
if (BINARY_CROSS_ENTROPY):
  loss_loaded = tf.keras.losses.binary_crossentropy
else if (BINARY_AND_DICE):
  loss_loaded = dice_and_binary_crossentropy
else if (BINARY_AND_JACCARD):
  loss_loaded = jaccard_and_binary_crossentropy

# F1-score metric 

In [ ]:
#We define here our F1 method
import keras
from keras import backend as K


def recall_m(y_true, y_pred):
    '''
    computes the recall
    source: https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
    :param y_true: real label
    :param y_pred: prediction
    :return recall: the computed recall
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    '''
    computes the precision
    source: https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
    :param y_true: real label
    :param y_pred: prediction
    :return recall: the computed precision
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    '''
    computes the F1_score
    source: https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
    :param y_true: real label
    :param y_pred: prediction
    :return : the computed f1_score
    '''
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# U-net model definition

In [ ]:
# Build U-Net model
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

conv1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
conv1 = BatchNormalization() (conv1)
conv1 = Dropout(0.1) (conv1)
conv1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv1)
conv1 = BatchNormalization() (conv1)
pooling1 = MaxPooling2D((2, 2)) (conv1)

conv2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling1)
conv2 = BatchNormalization() (conv2)
conv2 = Dropout(0.1) (conv2)
conv2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv2)
conv2 = BatchNormalization() (conv2)
pooling2 = MaxPooling2D((2, 2)) (conv2)

conv3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling2)
conv3 = BatchNormalization() (conv3)
conv3 = Dropout(0.2) (conv3)
conv3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv3)
conv3 = BatchNormalization() (conv3)
pooling3 = MaxPooling2D((2, 2)) (conv3)

conv4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling3)
conv4 = BatchNormalization() (conv4)
conv4 = Dropout(0.2) (conv4)
conv4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv4)
conv4 = BatchNormalization() (conv4)
pooling4 = MaxPooling2D(pool_size=(2, 2)) (conv4)

conv5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling4)
conv5 = BatchNormalization() (conv5)
conv5 = Dropout(0.3) (conv5)
conv5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv5)
conv5 = BatchNormalization() (conv5)


upsample6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (conv5)
upsample6 = concatenate([upsample6, conv4])
conv6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample6)
conv6 = BatchNormalization() (conv6)
conv6 = Dropout(0.2) (conv6)
conv6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv6)
conv6 = BatchNormalization() (conv6)

upsample7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (conv6)
upsample7 = concatenate([upsample7, conv3])
conv7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample7)
conv7 = BatchNormalization() (conv7)
conv7 = Dropout(0.2) (conv7)
conv7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv7)
conv7 = BatchNormalization() (conv7)

upsample8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (conv7)
upsample8 = concatenate([upsample8, conv2])
conv8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample8)
conv8 = BatchNormalization() (conv8)
conv8 = Dropout(0.1) (conv8)
conv8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv8)
conv8 = BatchNormalization() (conv8)

upsample9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (conv8)
upsample9 = concatenate([upsample9, conv1], axis=3)
conv9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample9)
conv9 = BatchNormalization() (conv9)
conv9 = Dropout(0.1) (conv9)
conv9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv9)
conv9 = BatchNormalization() (conv9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv9)


model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss=loss_loaded, metrics=['accuracy', f1_m]) #We compile our model using the loss we previously loaded
model.summary()




# Training or loading weights 

In [ ]:
#We create a model checkpoint to save our model
#We can either load it, or create it if necessary
checkpoint_path = data_dir + extraDirectory + ".ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
if (NEED_TO_CREATE_MODEL):
  # Create checkpoint callback
  cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                  save_weights_only=True,
                                                  verbose=1)

  callbacks = [
    cp_callback 
  ]

  results = model.fit(x_train, y_train, validation_split=VALIDATIONSPLIT, batch_size=BATCH_SIZE, epochs=EPOCHS,shuffle=True,
                      callbacks=callbacks)
else:
  model.load_weights(checkpoint_path)

# Create predictions 

In [ ]:
# We create here all the predictions using the model we just load/created and then compress
# and save it in case of future needs.
if (PREDICTION_IMAGE_EXIST):
  predictions = np.load(data_dir +'predictions'+ extraDirectory +'.npz')['arr_0']
else:
  dir = data_dir + 'images'
  list_ = os.listdir(dir) # dir is your directory path
  toSave = []
  for n, id_ in tqdm(enumerate(list_), total=len(list_)): 
    img = x_train[n]
    img =np.expand_dims(img, axis=0)
    predict = np.squeeze(model.predict(img, verbose=0))
    toSave.append(predict)

  predictions = toSave
  savez_compressed(data_dir +'predictions'+ extraDirectory+ '.npz', toSave)
  PREDICTION_IMAGE_EXIST = True

# Testing

In [ ]:
#We're here testing on an image to make sure our model works.

idx = 12
if (PREDICTION_IMAGE_EXIST):
  predict = predictions[idx]
else:   
  x=np.array(x_train[idx])
  x=np.expand_dims(x, axis=0)
  predict = np.squeeze(model.predict(x, verbose = "0"))

print("Prediction using our model :")
imshow(predict)

plt.show()

print("Value of the groundtruth")
imshow(np.squeeze(y_train[idx]))

plt.show()



# Image rotation and symmetry functions

In [ ]:
# This is an home made version to create rotations and fliped images. 
# For each image it will create 12 new others images, 4 rotations (0, 90, 180, and 270 degrees) 
# and 3 split (no split, split on X avis and split on Y axis)

def create_rotations(filename):
    '''
    Creates 3 rotations of a given image and on its 
    corresponding groundtruth by 90 degrees 
    :param filename: path of the image to rotate
    :return ouptput: list of the 8 images
    '''
    imageDirectory = originalDataImage + filename
    groundtruthDirectory = originalDataGrountruth + filename
    img = mpimg.imread(imageDirectory)
    groundtruth = mpimg.imread(groundtruthDirectory)
    imagesRotated = []
    groundTruthRotated = []
    output = []
    for i in range(0,4):
      imagesRotated.append(np.rot90(img,i))
      groundTruthRotated.append(np.rot90(groundtruth,i))
    output.append(imagesRotated)
    output.append(groundTruthRotated)
    return output

def create_symmetry(image, groundtruth_):
        '''
    Creates 2 symmetries of a given image and on its corresponding
    groundtruth on the x and y axis respectively 
    :param image: image to apply the symmetries on
    :param groundtruth_: image corresponding groundtruth 
    :return ouptput: list of  the 6 images
    '''
    img = image
    groundtruth = groundtruth_
    imagesSymmetry = []
    groundTruthSymmetry = []
    output = []
    imagesSymmetry.append(Image.fromarray((255.0 / img.max() * (img - img.min())).astype(np.uint8)))
    imagesSymmetry.append(Image.fromarray(((255.0 / np.flip(img, 0).max() * (np.flip(img, 0) - np.flip(img, 0).min())).astype(np.uint8))))
    imagesSymmetry.append(Image.fromarray((255.0 / np.flip(img, 1).max() * (np.flip(img, 1) - np.flip(img, 1).min())).astype(np.uint8)))

    groundTruthSymmetry.append(Image.fromarray((255.0 / groundtruth.max() * (groundtruth - groundtruth.min())).astype(np.uint8)))
    groundTruthSymmetry.append(Image.fromarray(((255.0 / np.flip(groundtruth, 0).max() * (np.flip(groundtruth, 0) - np.flip(groundtruth, 0).min())).astype(np.uint8))))
    groundTruthSymmetry.append(Image.fromarray((255.0 / np.flip(groundtruth, 1).max() * (np.flip(groundtruth, 1) - np.flip(groundtruth, 1).min())).astype(np.uint8)))
    output.append(imagesSymmetry)
    output.append(groundTruthSymmetry)
    return output

if not os.path.isdir(train_data_filename):
  os.mkdir(train_data_filename)
if not os.path.isdir(train_labels_filename):
  os.mkdir(train_labels_filename)
for i in tqdm(range(1, 101)):
        imageid = "satImage_%.3d" % i + '.png'
        [rotationsImg, groundTruthImg] = create_rotations(imageid )
        for k in range(0,4): 
          [symmetryImg, symmetryGroundTruth] = create_symmetry(rotationsImg[k],groundTruthImg[k])
          for j in range(0,3):
            imageidToSave = 'satImage_%4d' % ((i-1)*12+k*3+j) 
            symmetryImg[j].save(train_data_filename + imageidToSave + '.png')
            symmetryGroundTruth[j].save(train_labels_filename + imageidToSave + '.png')
        

# Submission creation

In [ ]:
#This part is to create a AICrown submission, the only parameter is the foreground_treshold

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename, index):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(index, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        index = 1
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn, index))
            index+=1



if __name__ == '__main__':
    submission_filename = data_dir + 'FinalResult.csv'
    image_filenames = []
    for i in tqdm(range(1, 51)):
      x = imread(test_dir_image +  str(i) + '.png' )[:,:,:IMG_CHANNELS]
      x_notsmooth = resize(x, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
      x_notsmooth=np.expand_dims(x_notsmooth, axis=0)
      prediction_not_smooth = model.predict(x_notsmooth, verbose=0)
      #We need to resize our image from (400,400) to (608,608)
      prediction_not_smooth = resize(prediction_not_smooth[0], (IMG_HEIGHT+208, IMG_WIDTH+208), mode='constant', preserve_range=True) 

      #This is the prediction that is resized using the cheat_tilting_prediction, it happend that the resize function had better resultst than this one.
      predictions_smooth = cheap_tiling_prediction(x, IMG_WIDTH, 1, pred_func=(
          lambda img_batch_subdiv: model.predict(np.expand_dims(img_batch_subdiv, axis=0)[:,0,:,:])
          )
      )

      image_filename = test_dir_groundtruth + '%.3d' % i + '.png'
      mpimg.imsave( image_filename, np.squeeze(prediction_not_smooth))
      image_filenames.append(image_filename)
    masks_to_submission(submission_filename, *image_filenames)

 

# Find best threshold

In [ ]:
#This part is made to find the best possible threshold value
MIN_VALUE = 0.1
MAX_VALUE = 0.9
STEP = 0.05
fg_array =  np.arange(MIN_VALUE,MAX_VALUE+STEP,STEP)
#!/usr/bin/env python3



errorArray = []
min = sys.float_info.max
index = 0
# assign a label to a patch
def patch_to_label(patch, f):
    df = np.mean(patch)
    if df > f:
        return 1
    else:
        return 0


def mask_to_submission_strings_2(prediction, f):
    """Reads a single image and outputs the strings that should go into the submission file"""
    toReturn = np.zeros((prediction.shape[0], prediction.shape[1]))
    patch_size = 16
    for j in range(0, prediction.shape[1], patch_size):
        for i in range(0, prediction.shape[0], patch_size):
            patch = prediction[i:i + patch_size, j:j + patch_size]
            toReturn[i:i + patch_size, j:j + patch_size] = patch_to_label(patch,f)
    return toReturn

image_filenames = []

if  (PREDICTION_IMAGE_EXIST==False):
  raise NameError('MAKE SURE TO FIRST CREATE THE PREDICTIONS USING THE MODEL OF THE IMAGES BEFORE TRYING TO FIND THE CORRECT THRESHOLD')  
else:
  for idx, f in tqdm(enumerate(fg_array), total= len(fg_array)):
    error = 0
    for i in (range(0, NUMBER_OF_FILE_TO_SEARSH+1)):
      predict = predictions[i]

      mask =  mask_to_submission_strings_2(predict,f)
      error += np.abs(mask - np.squeeze(np.where(y_train[i]==True, 1, 0))).sum() #Computing the error between the groundtruth and the prediction with the labels
    if (error<min):
      min = error
      index = idx
    errorArray.append(error/NUMBER_OF_FILE_TO_SEARSH)  
  plt.plot(fg_array, errorArray);
  print("Threshold value is : ")
  print(fg_array[index])

